In [103]:
import pandas as pd
import pyodbc
import sqlite3

# iedereen heeft andere servername btw

In [104]:
servername = 'LAPTOP-C1FMPSTV\\SQLEXPRESS01'

# Northwind connectie

In [105]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [106]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [107]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)}; DBQ=../raw-databases/aenc.accdb')
aencCursor = aenc_conn.cursor()

table_names = [table_info.table_name for table_info in aencCursor.tables()]

# Print table names
print(table_names)

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


AdventureWorks Person.Person / Person.Address / Person.PersonPhone

In [108]:
df_customer =  pd.read_sql_query('SELECT * FROM Sales.Customer', adventureworks_conn)
df_company = pd.read_sql_query('SELECT * FROM Sales.Store', adventureworks_conn)
df_person_address = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
df_business_entity_address = pd.read_sql_query('SELECT * FROM Person.BusinessEntityAddress', adventureworks_conn)
df_person_phone = pd.read_sql_query('SELECT * FROM Person.PersonPhone', adventureworks_conn)

df_company = df_company.rename(columns={"BusinessEntityID":"StoreID","Name":"company_name","SalesPersonID":"BusinessEntityID"})

aw_CC = pd.merge(df_customer[['CustomerID','StoreID','PersonID']], df_company[['BusinessEntityID','StoreID','company_name']], on='StoreID', how='left')
aw_cc_pisna = aw_CC.loc[aw_CC['PersonID'].isna(),['CustomerID','BusinessEntityID','company_name']]
aw_cp = aw_CC.loc[~aw_CC['PersonID'].isna(),['PersonID','CustomerID','company_name']]

aw_cc_pisna = aw_cc_pisna.rename(columns={"BusinessEntityID":"PersonID"})
aw_PC = pd.concat([aw_cc_pisna,aw_cp])
aw_custoemer = pd.merge(aw_PC, df_person_phone[['BusinessEntityID','PhoneNumber']],left_on='PersonID',right_on='BusinessEntityID', how='left')
aw_custoemer_adres = pd.merge(aw_custoemer,df_business_entity_address[['BusinessEntityID','AddressID']],on='BusinessEntityID', how='left')
aw_ca = pd.merge(aw_custoemer_adres,df_person_address,on='AddressID', how='left')
merged_df_aw = aw_ca

merged_df_aw

# BusinessEntityID	AddressLine1	City	PostalCode	PhoneNumber

C:\Users\sheng\AppData\Local\Temp\ipykernel_60276\1856417381.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customer =  pd.read_sql_query('SELECT * FROM Sales.Customer', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_60276\1856417381.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_company = pd.read_sql_query('SELECT * FROM Sales.Store', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_60276\1856417381.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_person_address = pd.

,CustomerID,PersonID,company_name,BusinessEntityID,PhoneNumber,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,SpatialLocation,rowguid,ModifiedDate
0,1,280.0,A Bike Store,280,340-555-0193,22.0,636 Vine Hill Way,None,Portland,58.0,97205,b'\xe6\x10\x00\x00\x01\x0c\xb5[\x85#\x18\xc3F@...,7F641525-2704-4F73-9D8A-EB062CFBFA3C,2011-05-24
1,2,283.0,Progressive Sports,283,740-555-0182,32507.0,2284 Azalea Avenue,None,Bellevue,79.0,98004,b'\xe6\x10\x00\x00\x01\x0c\xbb7\x1c\xdbl\xd2G@...,1C2AAEA8-0C55-4668-A40F-FA1F10766135,2011-05-24
2,3,277.0,Advanced Bike Components,277,517-555-0117,24.0,80 Sunview Terrace,None,Duluth,36.0,55802,b'\xe6\x10\x00\x00\x01\x0c\x04UJ}\xd5fG@Y\xf1\...,BE07D3C8-6E58-4670-9DA9-151AB6D3D620,2011-05-24
3,4,277.0,Modular Cycle Systems,277,517-555-0117,24.0,80 Sunview Terrace,None,Duluth,36.0,55802,b'\xe6\x10\x00\x00\x01\x0c\x04UJ}\xd5fG@Y\xf1\...,BE07D3C8-6E58-4670-9DA9-151AB6D3D620,2011-05-24
4,5,281.0,Metropolitan Sports Supply,281,330-555-0120,26.0,5725 Glaze Drive,None,San Francisco,9.0,94109,b'\xe6\x10\x00\x00\x01\x0c\xfe8\x83\xfdZ\xe1B@...,77415959-A4EE-4DA6-86F9-70C097560005,2011-05-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19839,30114,1985.0,Recreation Toy Store,1985,1 (11) 500 555-0116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
19840,30115,1987.0,Retreat Inn,1987,155-555-0140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
19841,30116,1989.0,Technical Parts Manufacturing,1989,433-555-0168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
19842,30117,1991.0,Totes & Baskets Company,1991,560-555-0171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


aenc customer

In [109]:
df_aenc_customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
df_aenc_customer = df_aenc_customer.rename(columns={"id":"CustomerID"})
df_aenc_customer

C:\Users\sheng\AppData\Local\Temp\ipykernel_60276\4165816717.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_aenc_customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)


,CustomerID,fname,lname,address,city,state,zip,phone,company_name
0,101,Michaels,Devlin,3114 Pioneer Avenue,Rutherford,NJ,07070,2015558966,The Power Group
1,102,Beth,Reiser,1033 Whippany Road,New York,NY,10154,2125558725,AMF Corp.
2,103,Erin,Niedringhaus,1990 Windsor Street,Paoli,PA,19301,2155556513,Darling Associates
3,104,Meghan,Mason,550 Dundas Street East,Knoxville,TN,37919,6155555463,P.S.C.
4,105,Laura,McCarthy,1210 Highway 36,Carmel,IN,46032,3175558437,Amo & Sons
...,...,...,...,...,...,...,...,...,...
121,552,Janice,O'Toole,654 West Hill,Nashville,TN,37320,6155553689,Greensleeves
122,553,Stevie,Nickolas,77 Recordings Circle,Tacoma,WA,96521,5095551695,It's a Hit!
123,555,Philipe,Fernandez,99 Main Street,Los Angeles,CA,90205,2135554457,Quaker Fashions
124,661,Jennifer,Stutzman,3 Back Pages Lane,Missola,IL,60505,7085556857,Stutzman Advertising


NorthWind customers

In [110]:
df_northwind_customers = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

C:\Users\sheng\AppData\Local\Temp\ipykernel_60276\2771955829.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_northwind_customers = pd.read_sql_query('SELECT * FROM customers', northwind_conn)


Specifieke Data eruit halen

In [111]:
# Select specific columns and rename them
df_aw_customers_selected = merged_df_aw.loc[:,['CustomerID', 'AddressLine1', 'City', 'PostalCode', 'PhoneNumber', 'company_name']]
df_northwind_customers_selected = df_northwind_customers.loc[:,['CustomerID', 'CompanyName', 'Address', 'City', 'PostalCode', 'Phone']]
df_aenc_customer_selected = df_aenc_customer.loc[:,['CustomerID', 'address', 'city', 'zip', 'phone', 'company_name']]

df_aw_customers_selected['Source'] = "AW"
df_northwind_customers_selected['Source'] = "NW"
df_aenc_customer_selected['Source'] = "AC"

df_aw_customers_selected

,CustomerID,AddressLine1,City,PostalCode,PhoneNumber,company_name,Source
0,1,636 Vine Hill Way,Portland,97205,340-555-0193,A Bike Store,AW
1,2,2284 Azalea Avenue,Bellevue,98004,740-555-0182,Progressive Sports,AW
2,3,80 Sunview Terrace,Duluth,55802,517-555-0117,Advanced Bike Components,AW
3,4,80 Sunview Terrace,Duluth,55802,517-555-0117,Modular Cycle Systems,AW
4,5,5725 Glaze Drive,San Francisco,94109,330-555-0120,Metropolitan Sports Supply,AW
...,...,...,...,...,...,...,...
19839,30114,NaN,NaN,NaN,1 (11) 500 555-0116,Recreation Toy Store,AW
19840,30115,NaN,NaN,NaN,155-555-0140,Retreat Inn,AW
19841,30116,NaN,NaN,NaN,433-555-0168,Technical Parts Manufacturing,AW
19842,30117,NaN,NaN,NaN,560-555-0171,Totes & Baskets Company,AW


Samenvoegen in Customer DF

In [112]:
# Concatenate the dataframes along axis 0 to stack them vertically
customer = pd.concat([df_northwind_customers_selected, df_aenc_customer_selected, df_aw_customers_selected], axis=0)

# Reassign values for columns as per the specified criteria
customer['customer_id'] = customer['CustomerID']
customer['customer_company_name'] = customer['company_name'].combine_first(customer['CompanyName'])
customer['customer_address'] = customer['address'].combine_first(customer['Address']).combine_first(customer['AddressLine1'])
customer['customer_city'] = customer['city'].combine_first(customer['City'])
customer['customer_zip'] = customer['PostalCode'].combine_first(customer['zip'])
customer['customer_phone'] = customer['phone'].combine_first(customer['PhoneNumber']).combine_first(customer['Phone'])
customer['source'] = customer['Source']

# Drop redundant columns
customer.drop(['CustomerID', 'CompanyName', 'Address', 'AddressLine1', 'City', 'PostalCode', 'PhoneNumber', 'Phone'], axis=1, inplace=True)

# Reorder the columns
customer = customer[['customer_id', 'customer_company_name', 'customer_address', 'customer_city', 'customer_zip', 'customer_phone','source']]

# Display where id is 101
customer=customer.where(pd.notna(customer), None)
customer

,customer_id,customer_company_name,customer_address,customer_city,customer_zip,customer_phone,source
0,ALFKI,Alfreds Futterkiste,Obere Str. 57,Berlin,12209,030-0074321,NW
1,ANATR,Ana Trujillo Emparedados y helados,Avda. de la Constitución 2222,México D.F.,05021,(5) 555-4729,NW
2,ANTON,Antonio Moreno Taquería,Mataderos 2312,México D.F.,05023,(5) 555-3932,NW
3,AROUT,Around the Horn,120 Hanover Sq.,London,WA1 1DP,(171) 555-7788,NW
4,BERGS,Berglunds snabbk?p,Berguvsv?gen 8,Lule?,S-958 22,0921-12 34 65,NW
...,...,...,...,...,...,...,...
19839,30114,Recreation Toy Store,None,None,None,1 (11) 500 555-0116,AW
19840,30115,Retreat Inn,None,None,None,155-555-0140,AW
19841,30116,Technical Parts Manufacturing,None,None,None,433-555-0168,AW
19842,30117,Totes & Baskets Company,None,None,None,560-555-0171,AW


Convert types to varchar

In [113]:

customer.dtypes

customer_id              object
customer_company_name    object
customer_address         object
customer_city            object
customer_zip             object
customer_phone           object
source                   object
dtype: object

In [114]:
united_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=UnitedOutdoors; Trusted_Connection=yes')
unitedCursor = united_conn.cursor()
unitedCursor

In [115]:
for index, row in customer.iterrows():
    try:
        query = """INSERT INTO customer (
            customer_id,
            customer_company_name, 
            customer_address, 
            customer_city,
            customer_zip,
            customer_phone,
            customer_source
        ) VALUES (?, ?, ?, ?, ?, ?, ?)"""  
        
        values = (
            row['customer_id'],
            row['customer_company_name'],
            row['customer_address'],
            row['customer_city'],
            row['customer_zip'],
            row['customer_phone'],
            row['source']
        )

        unitedCursor.execute(query, values)

    except pyodbc.IntegrityError as e:
        continue
    except pyodbc.Error as e:
        print("Er is een fout opgetreden: ", e)
        print("Foutieve query: ", query)
    
unitedCursor.commit()

print("Data is succesvol geïmporteerd in de UnitedOutdoors database!")

Data is succesvol geïmporteerd in de UnitedOutdoors database


In [116]:
aenc_conn.close()